In [19]:
import numpy as np
import pickle as pkl

import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data_frames = pkl.load(open('full_dataset_frames.pkl', 'rb'))
data_events = pkl.load(open('full_dataset_events.pkl', 'rb'))

print("==== FRAMES ====")
print(f"video -> {data_frames['video'].shape}")
print(f"labels -> {data_frames['labels'].shape}")
print(f"emg -> {data_frames['emg'].shape}")

print("==== EVENTS ====")
print(f"video -> {data_events['video'].shape}")
print(f"labels -> {data_events['labels'].shape}")
print(f"emg -> {data_events['emg'].shape}")

==== FRAMES ====
video -> (5800, 1296)
labels -> (5800,)
emg -> (5800, 24)
==== EVENTS ====
video -> (6000, 1296)
labels -> (6000,)
emg -> (6000, 24)


In [15]:
def single_svm(x, y):
    
    idx = np.random.permutation(len(x))
    n_train = int(len(x) * 0.8)
    idx_train = idx[:n_train]
    idx_test = idx[n_train:]


    x_train = x[idx_train]
    y_train = y[idx_train]

    x_test = x[idx_test]
    y_test = y[idx_test]
    
    x_mean = np.mean(x_train, 0, keepdims=True)
    x_std = np.std(x_train, 0, keepdims=True)

    x_train = (x_train - x_mean) / x_std
    x_test = (x_test - x_mean) / x_std

    # Train the SVM for demg:
    svm = cv2.ml.SVM_create()
    svm.setType(cv2.ml.SVM_C_SVC)
    svm.setKernel(cv2.ml.SVM_LINEAR)
    svm.setC(0.01)
    svm.setGamma(2)
    svm.train(np.float32(x_train), cv2.ml.ROW_SAMPLE, np.int32(y_train))

    output = svm.predict(np.float32(x_train))
    acc = np.sum((output[1].squeeze() - y_train.squeeze()) == 0) / len(output[1])
    print("Train: {:.4}".format(acc))

    output = svm.predict(np.float32(x_test))
    acc = np.sum((output[1].squeeze() - y_test.squeeze()) == 0) / len(output[1])
    print("Test: {:.4}".format(acc))

In [17]:
print("Frames")
single_svm(data_frames['video'], data_frames['labels'])
print("Emg")
single_svm(data_frames['emg'], data_frames['labels'])
print("Events")
single_svm(data_events['video'], data_events['labels'])
print("Frames+Emg")
single_svm(np.hstack([data_frames['emg'], data_frames['video']]), data_frames['labels'])
print("Events+Emg")
single_svm(np.hstack([data_events['emg'], data_events['video']]), data_events['labels'])

Frames
Train: 0.8929
Test: 0.7
Emg
Train: 0.5614
Test: 0.5793
Events
Train: 0.9348
Test: 0.7092
Frames+Emg
Train: 0.9422
Test: 0.8216
Events+Emg
Train: 0.9917
Test: 0.82
